In [1]:
import os
import pandas as pd
import numpy as np
import xlsxwriter

In [37]:
experiments = ['IEEE9_currents_raw_375', 
               'IEEE9_currents_raw_505', 
               'IEEE9_currents_raw_3red_375',
               'IEEE9_currents_raw_3red_505',
               'IEEE9_noiseless_raw_375',
               'IEEE9_noiseless_raw_505',
               'IEEE9_noiseless_raw_3red_375',
               'IEEE9_noiseless_raw_3red_505',
               'IEEE9_noisy_raw_375',
               'IEEE9_noisy_raw_505',
               'IEEE9_noisy_raw_3red_375',
               'IEEE9_noisy_raw_3red_505']

architectures = ['fcn', 'inception', 'mcdcnn', 'tlenet', 'mlp', 'cnn']

## Normal Experiments

In [63]:
df_experiments = dict.fromkeys(experiments)
architectures = ['fcn', 'inception', 'mcdcnn', 'tlenet', 'mlp', 'cnn']

for experiment in experiments:
    df_res_arch = dict.fromkeys(architectures)
    for architecture in architectures:
        path_res = os.path.join(os.getcwd(), 'experiments', experiment, architecture)
        path_best = os.path.join(path_res, '_df_best_model.csv')
        
        # Loading the best model history report
        df_res_best = pd.read_csv(path_best)
        
        # Extracting the best model epoch
        _epoch_best = df_res_best['best_model_nb_epoch'].values[0]
        
        path_results = os.path.join(path_res, 'results_normal.csv')
        df_res = pd.read_csv(path_results)
        
        _prec = df_res['precision'].values[0]
        _acc = df_res['accuracy'].values[0]
        _rec = df_res['recall'].values[0]
        _f1 = df_res['f1'].values[0]
        _pred_time = df_res['prediction_time'].values[0]
        
        df_res_arch[architecture] = {'f1': _f1, 
         'accuracy' : _acc, 
         'precision' :  _prec, 
         'recall' : _rec, 
         'prediction_time' : _pred_time, 
        'epoch_best' : _epoch_best}
        
        df_res = None
        df_res_best = None
        
    df_experiments[experiment] = df_res_arch
    
    df_res_arch = None

In [65]:
workbook = xlsxwriter.Workbook('results_normal.xlsx')
ws_experiments = dict.fromkeys(experiments)

cell_format = workbook.add_format({'bold': True, 'font_name': 'Times', 'align' : 'center'})

for experiment in experiments:
    ws_experiments[experiment] = workbook.add_worksheet(experiment)
    ws_experiments[experiment].write('B1', 'Model', cell_format)
    ws_experiments[experiment].write('C1', 'F1', cell_format)
    ws_experiments[experiment].write('D1', 'Accuracy', cell_format)
    ws_experiments[experiment].write('E1', 'Precision', cell_format)
    ws_experiments[experiment].write('F1', 'Recall', cell_format)
    ws_experiments[experiment].write('G1', 'Prediction Time', cell_format)
    ws_experiments[experiment].write('H1', 'Epoch Best', cell_format)
    
cell_format = workbook.add_format({'font_name': 'Times'})

for experiment in experiments:
    for n, architecture in enumerate(architectures):
        ws_experiments[experiment].write(f'B{n+2}', architecture, cell_format)
        
        ws_experiments[experiment].write_number(n+1, 2, df_experiments[experiment][architecture]['f1'], cell_format)
        ws_experiments[experiment].write_number(n+1, 3, df_experiments[experiment][architecture]['accuracy'], cell_format)
        ws_experiments[experiment].write_number(n+1, 4, df_experiments[experiment][architecture]['precision'], cell_format)
        ws_experiments[experiment].write_number(n+1, 5, df_experiments[experiment][architecture]['recall'], cell_format)
        ws_experiments[experiment].write_number(n+1, 6, df_experiments[experiment][architecture]['prediction_time'], cell_format)
        ws_experiments[experiment].write_number(n+1, 7, df_experiments[experiment][architecture]['epoch_best'], cell_format)
        
workbook.close()

## k-fold

In [43]:
df_experiments = dict.fromkeys(experiments)

for experiment in experiments:
    df_res_arch = dict.fromkeys(architectures)
    for architecture in architectures:
        path_res = os.path.join(os.getcwd(), 'experiments', experiment, architecture, 'k-fold')
        
        epoch_best = 0
        for n in range(1, 6):
            path_best = os.path.join(path_res, f'{n}_df_best_model.csv')
            df_path_best = pd.read_csv(path_best)
            epoch_best += df_path_best['best_model_nb_epoch'].values[0]
        
        epoch_best /= n
        epoch_best = int(epoch_best)
        
        path_res = os.path.join(os.getcwd(), 'experiments', experiment, architecture, 'k-fold', 'results_k-fold.csv')
        df_res = pd.read_csv(path_res)
        
        f1_mean = df_res['F1'].mean() * 100
        f1_std = df_res['F1'].std() * 100
        
        acc_mean = df_res['accuracy'].mean() * 100
        acc_std = df_res['accuracy'].std() * 100
        
        prec_mean = df_res['precision'].mean() * 100
        prec_std = df_res['precision'].std() * 100
        
        rec_mean = df_res['recall'].mean() * 100
        rec_std = df_res['recall'].std() * 100
        
        pred_t_mean = df_res['prediction_time'].mean()
        pred_t_std = df_res['prediction_time'].std()
        
        df_res_arch[architecture] = {'f1': [f1_mean, f1_std], 
         'accuracy' : [acc_mean, acc_std], 
         'precision' :  [prec_mean, prec_std], 
         'recall' : [rec_mean, rec_std], 
         'prediction_time' : [pred_t_mean, pred_t_std], 
        'epoch_best' : epoch_best}
        
        df_res = None
        df_path_best = None
    
    df_experiments[experiment] = df_res_arch
    df_res_arch = None

In [61]:
workbook = xlsxwriter.Workbook('results_k-fold.xlsx')
ws_experiments = dict.fromkeys(experiments)

cell_format = workbook.add_format({'bold': True, 'font_name': 'Times', 'align' : 'center'})

for experiment in experiments:
    ws_experiments[experiment] = workbook.add_worksheet(experiment)
    ws_experiments[experiment].merge_range('B1:B2', 'Model', cell_format)
    ws_experiments[experiment].merge_range('C1:D1', 'F1', cell_format)
    ws_experiments[experiment].merge_range('E1:F1', 'Accuracy', cell_format)
    ws_experiments[experiment].merge_range('G1:H1', 'Precision', cell_format)
    ws_experiments[experiment].merge_range('I1:J1', 'Recall', cell_format)
    ws_experiments[experiment].merge_range('K1:L1', 'Prediction Time', cell_format)
    ws_experiments[experiment].merge_range('M1:M2', 'Epoch Best', cell_format)
    
    ws_experiments[experiment].write('C2', 'Mean', cell_format)
    ws_experiments[experiment].write('D2', 'Std', cell_format)
    ws_experiments[experiment].write('E2', 'Mean', cell_format)
    ws_experiments[experiment].write('F2', 'Std', cell_format)
    ws_experiments[experiment].write('G2', 'Mean', cell_format)
    ws_experiments[experiment].write('H2', 'Std', cell_format)
    ws_experiments[experiment].write('I2', 'Mean', cell_format)
    ws_experiments[experiment].write('J2', 'Std', cell_format)
    ws_experiments[experiment].write('K2', 'Mean', cell_format)
    ws_experiments[experiment].write('L2', 'Std', cell_format)

cell_format = workbook.add_format({'font_name': 'Times'})
    
for experiment in experiments:
    for n, architecture in enumerate(architectures):
        ws_experiments[experiment].write(f'B{n+3}', architecture, cell_format)
        
        ws_experiments[experiment].write_number(n+2, 2, df_experiments[experiment][architecture]['f1'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 3, df_experiments[experiment][architecture]['f1'][1], cell_format)
        ws_experiments[experiment].write_number(n+2, 4, df_experiments[experiment][architecture]['accuracy'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 5, df_experiments[experiment][architecture]['accuracy'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 6, df_experiments[experiment][architecture]['precision'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 7, df_experiments[experiment][architecture]['precision'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 8, df_experiments[experiment][architecture]['recall'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 9, df_experiments[experiment][architecture]['recall'][1], cell_format)
        ws_experiments[experiment].write_number(n+2, 10, df_experiments[experiment][architecture]['prediction_time'][0], cell_format)
        ws_experiments[experiment].write_number(n+2, 11, df_experiments[experiment][architecture]['prediction_time'][1], cell_format)
        ws_experiments[experiment].write_number(n+2, 12, df_experiments[experiment][architecture]['epoch_best'], cell_format)
        
workbook.close()